# Vector Database with Approximate Nearest Neighbors

In [30]:
from transformers import AutoTokenizer, AutoModel
import torch

from random import random, randint
from math import floor, log
import numpy as np
import networkx as nx
import PyPDF2
from matplotlib import pyplot as plt
from utils import layer_num, construct_HNSW, search_HNSW

np.random.seed(44)

m_nearest_neighbor = 3 # M Nearest Neigbor used in construction of the Navigable Small World (NSW)

In [3]:
## Queries

np.random.seed(44)

with open('eval_questions.txt', 'r') as f:
    questions = list(map(lambda x: x.strip(), f.readlines()))

print(questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?']
nodes =  [('Q', {'pos': [0.5, 0.5]})]


In [4]:
# creating a pdf reader object
reader = PyPDF2.PdfReader('eBook-How-to-Build-a-Career-in-AI.pdf')

# print the number of pages in pdf file
print(len(reader.pages))

# print the text of the first page
print(reader.pages[0].extract_text())

41
PAGE 1Founder, DeepLearning.AICollected Insights
from Andrew Ng
How to 
Build
Your
Career
in AIA Simple Guide



In [5]:
pages = [{'text':text.extract_text(), 'i':i}  for i, text in enumerate(reader.pages)]

# sentences = [i.extract_text() for i in reader.pages].split('\n')

In [6]:
pages = list(map(lambda x: x['text'][0:6] + " " + x['text'][6:] 
                    if x['i']<9 
                    else x['text'][0:7] + " " + x['text'][7:], pages))[:-1]

In [7]:
pages

['PAGE 1 Founder, DeepLearning.AICollected Insights\nfrom Andrew Ng\nHow to \nBuild\nYour\nCareer\nin AIA Simple Guide\n',
 'PAGE 2 "AI is the new \nelectricity. It will \ntransform and improve \nall areas of human life."\nAndrew Ng',
 'PAGE 3 Table of \nContentsIntroduction: Coding AI is the New Literacy.\nChapter 1: Three Steps to Career Growth.\nChapter 2: Learning Technical Skills for a \nPromising AI Career.\nChapter 3: Should You Learn Math to Get a Job \nin AI?\nChapter 4: Scoping Successful AI Projects.\nChapter 5: Finding Projects that Complement \nYour Career Goals.\nChapter 6: Building a Portfolio of Projects that \nShows Skill Progression.\nChapter 7: A Simple Framework for Starting Your AI \nJob Search.\nChapter 8: Using Informational Interviews to Find \nthe Right Job.\nChapter 9: Finding the Right AI Job for You.\nChapter 10: Keys to Building a Career in AI.\nChapter 11: Overcoming Imposter Syndrome.\nFinal Thoughts: Make Every Day Count.LEARNING\nPROJECTS\nJOB',
 'PAGE 

In [8]:
def split_string_by_dot(s, max_length=500):
    
    segments = []
    while s:
        # If the remaining string is shorter than the max length, add it as a segment
        if len(s) <= max_length:
            segments.append(s)
            break

        # Find the first dot after the max length
        split_index = s.find('.', max_length - 1)

        # If no dot is found, split at max length
        if split_index == -1:
            split_index = max_length

        # Add the segment up to the dot
        segments.append(s[:split_index + 1])

        # Remove the processed part from the string
        s = s[split_index + 1:]

    return segments

In [21]:
new_pages = []
for i, page in enumerate(pages):

    new_pages += split_string_by_dot(page, max_length=500)

In [22]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

for i, page in enumerate(pages):
    new_pages += split_string_by_dot(page, max_length=500)

# Sentences we want sentence embeddings for
sentences = new_pages[:]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, average pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])b

In [39]:
encoded_input = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, average pooling
question_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [40]:
question_embeddings

tensor([[-0.0808,  0.2511, -0.0033,  ...,  0.0285,  0.0402, -0.1637],
        [-0.0294,  0.0611, -0.0006,  ...,  0.0296,  0.1894, -0.1427],
        [-0.0926, -0.0992, -0.0035,  ..., -0.0591, -0.0235, -0.1029],
        ...,
        [-0.1327,  0.0758, -0.0017,  ...,  0.0203, -0.0030, -0.1010],
        [ 0.0602, -0.0158, -0.0015,  ...,  0.0094,  0.0026, -0.1364],
        [ 0.0866,  0.0617,  0.0008,  ...,  0.0638,  0.0533, -0.1461]])

In [41]:
querries = []
for q in question_embeddings:
    query_vec = q
    nodes = []
    nodes.append(("Q",{"pos": query_vec}))

    G_query = nx.Graph()
    G_query.add_nodes_from(nodes)

    querries.append(G_query)

In [26]:
# database

texts = {}
embeddings = {}
for i in range(len(new_pages)):
    texts[i] = new_pages[i]
    embeddings[i] = sentence_embeddings[i]


In [31]:
# Construct Hierarchical Navigable Small World

graph_array = construct_HNSW(sentence_embeddings,m_nearest_neighbor)

In [34]:
questions[0]

'What are the keys to building a career in AI?'

In [42]:
search_HNSW(graph_array, querries[0])

([<networkx.classes.graph.Graph at 0x22f800d8b90>,
  <networkx.classes.graph.Graph at 0x22fee73d650>])

In [47]:
(SearchPathGraphArray, EntryGraphArray) = search_HNSW(graph_array, querries[0])

for layer_i in range(len(graph_array)-1,-1,-1):
#     fig, axs = plt.subplots()

    print("layer_i = ", layer_i)
    G_path_layer = SearchPathGraphArray[layer_i]
    pos_path = nx.get_node_attributes(G_path_layer,'pos')
    G_entry = EntryGraphArray[layer_i]
    pos_entry = nx.get_node_attributes(G_entry,'pos')

    if layer_i>0:
            pos_layer_0 = nx.get_node_attributes(graph_array[0],'pos')
        #     nx.draw(graph_array[0], pos_layer_0, with_labels=True, node_size=120, node_color=[[0.9,0.9,1]], width=0.0, font_size=6, font_color=(0.65,0.65,0.65), ax = axs)

    pos_layer_i = nx.get_node_attributes(graph_array[layer_i],'pos')

layer_i =  3
layer_i =  2
layer_i =  1
layer_i =  0


In [52]:
pos_layer_i[0]

tensor([-1.0212e-01,  2.5622e-01,  4.4929e-04, -1.6498e-02,  1.3855e-01,
         3.4664e-02,  2.7610e-01,  1.6154e-02,  1.2858e-01,  4.1059e-02,
         5.6357e-02, -3.1424e-03,  3.9346e-02,  3.2918e-01,  4.6758e-02,
        -9.1209e-02,  2.8952e-02,  5.2917e-02,  1.0886e-01,  7.7772e-03,
         9.0886e-02, -8.9739e-02,  3.6959e-02,  4.6793e-02, -1.1103e-01,
        -1.5419e-01, -5.5510e-02,  5.3616e-02,  5.8222e-02, -3.3153e-03,
        -1.3070e-02, -1.1830e-01,  9.2420e-02,  1.5508e-03,  5.8824e-02,
        -2.9443e-02,  1.9708e-04,  7.1338e-02, -1.5566e-01,  1.7488e-02,
        -8.8254e-03,  1.4613e-01,  6.9975e-02,  4.5326e-02, -1.2935e-01,
         1.4396e-01, -3.5398e-02,  1.1899e-01,  1.3616e-01,  1.0760e-01,
         1.5138e-02,  2.6700e-03,  1.2787e-01, -4.0584e-02, -5.6795e-02,
        -3.4698e-01, -5.6230e-02,  1.6619e-02, -2.2241e-01, -6.4874e-02,
        -7.3002e-02,  3.9570e-02, -1.0246e-02,  2.0094e-02,  8.3849e-02,
         1.3321e-02,  1.0163e-01, -1.0974e-01,  4.6

In [32]:
search_HNSW()

In [4]:
max_layers = 4

vec_num = np.shape(vec_pos)[0]
dist_mat = np.zeros((vec_num,vec_num))

In [11]:
for i in range(vec_num):
    for j in range(i,vec_num):
        print('i=='+str(i))
        print('j=='+str(j))
        print('i vec_pos', vec_pos[i,:])
        print('j vec_pos', vec_pos[j,:])
        print('div vec_pos', vec_pos[i,:]-vec_pos[j,:])
        print('linalg.norm vec_pos', np.linalg.norm(vec_pos[i,:]-vec_pos[j,:]))
        dist = np.linalg.norm(vec_pos[i,:]-vec_pos[j,:])
        dist_mat[i,j] = dist
        dist_mat[j,i] = dist

i==0
j==0
i vec_pos [0.83484215 0.1047961 ]
j vec_pos [0.83484215 0.1047961 ]
div vec_pos [0. 0.]
linalg.norm vec_pos 0.0
i==0
j==1
i vec_pos [0.83484215 0.1047961 ]
j vec_pos [0.74464048 0.36050084]
div vec_pos [ 0.09020167 -0.25570473]
linalg.norm vec_pos 0.2711480234782855
i==0
j==2
i vec_pos [0.83484215 0.1047961 ]
j vec_pos [0.35931084 0.60923838]
div vec_pos [ 0.47553131 -0.50444228]
linalg.norm vec_pos 0.6932474577468525
i==0
j==3
i vec_pos [0.83484215 0.1047961 ]
j vec_pos [0.39377955 0.40907261]
div vec_pos [ 0.4410626  -0.30427651]
linalg.norm vec_pos 0.5358361753143902
i==0
j==4
i vec_pos [0.83484215 0.1047961 ]
j vec_pos [0.50990241 0.71014799]
div vec_pos [ 0.32493974 -0.60535189]
linalg.norm vec_pos 0.6870493020845458
i==0
j==5
i vec_pos [0.83484215 0.1047961 ]
j vec_pos [0.96052623 0.45662111]
div vec_pos [-0.12568408 -0.351825  ]
linalg.norm vec_pos 0.37360048305073357
i==0
j==6
i vec_pos [0.83484215 0.1047961 ]
j vec_pos [0.42765152 0.1134637 ]
div vec_pos [ 0.40719063

In [23]:
node_layer = []
for i in range(np.shape(vec_pos)[0]):
    node_layer.append(layer_num(max_layers))

In [24]:
node_layer

[3, 1, 0, 2, 3, 0, 0, 2, 1, 0, 1, 0, 0, 3, 0]

In [ ]:
max_num_of_layers = max(node_layer) + 1 ## layer indices start from 0

In [26]:
pos

{0: array([0.83484215, 0.1047961 ]),
 1: array([0.74464048, 0.36050084]),
 2: array([0.35931084, 0.60923838]),
 3: array([0.39377955, 0.40907261]),
 4: array([0.50990241, 0.71014799]),
 5: array([0.96052623, 0.45662111]),
 6: array([0.42765152, 0.1134637 ]),
 7: array([0.21789887, 0.95747207]),
 8: array([0.94335072, 0.88182428]),
 9: array([0.64641056, 0.21382481]),
 10: array([0.63683201, 0.13914625]),
 11: array([0.45870407, 0.87386319]),
 12: array([0.25845031, 0.6648511 ]),
 13: array([0.86267434, 0.14884806]),
 14: array([0.56294996, 0.15915526])}

In [27]:
nodes

[(0, {'pos': array([0.83484215, 0.1047961 ])}),
 (1, {'pos': array([0.74464048, 0.36050084])}),
 (2, {'pos': array([0.35931084, 0.60923838])}),
 (3, {'pos': array([0.39377955, 0.40907261])}),
 (4, {'pos': array([0.50990241, 0.71014799])}),
 (5, {'pos': array([0.96052623, 0.45662111])}),
 (6, {'pos': array([0.42765152, 0.1134637 ])}),
 (7, {'pos': array([0.21789887, 0.95747207])}),
 (8, {'pos': array([0.94335072, 0.88182428])}),
 (9, {'pos': array([0.64641056, 0.21382481])}),
 (10, {'pos': array([0.63683201, 0.13914625])}),
 (11, {'pos': array([0.45870407, 0.87386319])}),
 (12, {'pos': array([0.25845031, 0.6648511 ])}),
 (13, {'pos': array([0.86267434, 0.14884806])}),
 (14, {'pos': array([0.56294996, 0.15915526])})]

In [44]:
max_num_of_layers = max(node_layer) + 1 ## layer indices start from 0
GraphArray = []
for layer_i in range(max_num_of_layers):
    nodes = []
    edges = []
    edges_nn = []
    for i in range(np.shape(vec_pos)[0]): ## Number of Vectors
        if node_layer[i] >= layer_i:
            nodes.append((i,{"pos": vec_pos[i,:]}))

    G = nx.Graph()
    G.add_nodes_from(nodes)

    pos=nx.get_node_attributes(G,'pos')

    for i in range (len(G.nodes)):
        node_i = nodes[i][0]
        nearest_edges = -1
        nearest_distances = float('inf')
        candidate_edges = range(0,i)
        candidate_edges_indices = []
        
        #######################
        for j in candidate_edges:
            node_j = nodes[j][0]
            candidate_edges_indices.append(node_j)
        
        dist_from_node = dist_mat[node_i,candidate_edges_indices]
        num_nearest_neighbor = min(m_nearest_neighbor,i) ### Add note comment
        
        if num_nearest_neighbor > 0:
            indices = np.argsort(dist_from_node)
            for nn_i in range(num_nearest_neighbor):
                    edges_nn.append((node_i,candidate_edges_indices[indices[nn_i]]))
        
        for j in candidate_edges:
            node_j = nodes[j][0]            
            dist = np.linalg.norm(pos[node_i]-pos[node_j])
            if dist < nearest_distances:
                nearest_edges = node_j
                nearest_distances = dist
        
        if nearest_edges != -1:
            edges.append((node_i,nearest_edges))

    G.add_edges_from(edges_nn)

    GraphArray.append(G)
    break

In [45]:
G.edges

EdgeView([(0, 1), (0, 2), (0, 5), (0, 9), (0, 10), (0, 13), (1, 2), (1, 3), (1, 5), (1, 6), (1, 9), (2, 3), (2, 4), (2, 7), (2, 12), (3, 4), (3, 6), (4, 7), (4, 8), (4, 11), (4, 12), (5, 8), (7, 11), (9, 10), (9, 13), (9, 14), (10, 14)])

In [46]:
G.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14))

In [ ]:
G.

In [39]:
len(candidate_edges_indices)

107

In [37]:
dist_mat[node_i,candidate_edges_indices]

array([0.27727294, 0.55354045, 0.27727294, 0.27727294, 0.27120377,
       0.27727294, 0.27120377, 0.4940078 , 0.27727294, 0.27120377,
       0.4940078 , 0.30179017, 0.27727294, 0.27120377, 0.4940078 ,
       0.30179017, 0.55354045, 0.27727294, 0.27120377, 0.4940078 ,
       0.30179017, 0.55354045, 0.49654085, 0.27727294, 0.27120377,
       0.4940078 , 0.30179017, 0.55354045, 0.49654085, 0.14280541,
       0.27727294, 0.27120377, 0.4940078 , 0.30179017, 0.55354045,
       0.49654085, 0.14280541, 0.86969533, 0.27727294, 0.27120377,
       0.4940078 , 0.30179017, 0.55354045, 0.49654085, 0.14280541,
       0.86969533, 0.81667328, 0.27727294, 0.27120377, 0.4940078 ,
       0.30179017, 0.55354045, 0.49654085, 0.14280541, 0.86969533,
       0.81667328, 0.0997719 , 0.27727294, 0.27120377, 0.4940078 ,
       0.30179017, 0.55354045, 0.49654085, 0.14280541, 0.86969533,
       0.81667328, 0.0997719 , 0.07654357, 0.27727294, 0.27120377,
       0.4940078 , 0.30179017, 0.55354045, 0.49654085, 0.14280

In [36]:
candidate_edges_indices

[0,
 4,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 3,
 4,
 0,
 1,
 2,
 3,
 4,
 5,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13]

In [34]:
node_i

14

In [33]:
candidate_edges

range(0, 14)

In [30]:
len(G.nodes)

3

In [29]:
candidate_edges_indices

[0, 4]

In [ ]:
def construct_HNSW(vec_pos,m_nearest_neighbor):
    max_layers = 4

    vec_num = np.shape(vec_pos)[0]
    dist_mat = np.zeros((vec_num,vec_num))

    for i in range(vec_num):
        for j in range(i,vec_num):
            dist = np.linalg.norm(vec_pos[i,:]-vec_pos[j,:])
            dist_mat[i,j] = dist
            dist_mat[j,i] = dist

    node_layer = []
    for i in range(np.shape(vec_pos)[0]):
        node_layer.append(layer_num(max_layers))
        
    max_num_of_layers = max(node_layer) + 1 ## layer indices start from 0
    GraphArray = []
    for layer_i in range(max_num_of_layers):
        nodes = []
        edges = []
        edges_nn = []
        for i in range(np.shape(vec_pos)[0]): ## Number of Vectors
            if node_layer[i] >= layer_i:
                nodes.append((i,{"pos": vec_pos[i,:]}))

        G = nx.Graph()
        G.add_nodes_from(nodes)

        pos=nx.get_node_attributes(G,'pos')

        for i in range (len(G.nodes)):
            node_i = nodes[i][0]
            nearest_edges = -1
            nearest_distances = float('inf')
            candidate_edges = range(0,i)
            candidate_edges_indices = []
            
            #######################
            for j in candidate_edges:
                node_j = nodes[j][0]
                candidate_edges_indices.append(node_j)
            
            dist_from_node = dist_mat[node_i,candidate_edges_indices]
            num_nearest_neighbor = min(m_nearest_neighbor,i) ### Add note comment
            
            if num_nearest_neighbor > 0:
                indices = np.argsort(dist_from_node)
                for nn_i in range(num_nearest_neighbor):
                        edges_nn.append((node_i,candidate_edges_indices[indices[nn_i]]))
            
            for j in candidate_edges:
                node_j = nodes[j][0]            
                dist = np.linalg.norm(pos[node_i]-pos[node_j])
                if dist < nearest_distances:
                    nearest_edges = node_j
                    nearest_distances = dist
            
            if nearest_edges != -1:
                edges.append((node_i,nearest_edges))

        G.add_edges_from(edges_nn)

        GraphArray.append(G)
        
    return GraphArray

In [ ]:
np.random.seed(44)

def nearest_neigbor(vec_pos,query_vec):
    nearest_neighbor_index = -1
    nearest_dist = float('inf')

    nodes = []
    edges = []
    for i in range(np.shape(vec_pos)[0]):
        nodes.append((i,{"pos": vec_pos[i,:]}))
        if i<np.shape(vec_pos)[0]-1:
            edges.append((i,i+1))
        else:
            edges.append((i,0))

        dist = np.linalg.norm(query_vec-vec_pos[i])
        if dist < nearest_dist:
            nearest_neighbor_index = i
            nearest_dist = dist
        
    G_lin = nx.Graph()
    G_lin.add_nodes_from(nodes)
    G_lin.add_edges_from(edges)

    nodes = []
    nodes.append(("*",{"pos": vec_pos[nearest_neighbor_index,:]}))
    G_best = nx.Graph()
    G_best.add_nodes_from(nodes)
    return G_lin, G_best

## Search the Graph
def search_HNSW(GraphArray,G_query):
    max_layers = len(GraphArray)
    G_top_layer = GraphArray[max_layers - 1]
    num_nodes = G_top_layer.number_of_nodes()
    entry_node_r = randint(0,num_nodes-1)
    nodes_list = list(G_top_layer.nodes)
    entry_node_index = nodes_list[entry_node_r]
    #entry_node_index = 26

    SearchPathGraphArray = []
    EntryGraphArray = []
    for l_i in range(max_layers):
        layer_i = max_layers - l_i - 1
        G_layer = GraphArray[layer_i]
        
        G_entry = nx.Graph()
        nodes = []
        p = G_layer.nodes[entry_node_index]['pos']
        nodes.append((entry_node_index,{"pos": p}))
        G_entry.add_nodes_from(nodes)
        
        nearest_node_layer = entry_node_index
        nearest_distance_layer = np.linalg.norm( G_layer.nodes[entry_node_index]['pos'] - G_query.nodes['Q']['pos'])
        current_node_index = entry_node_index

        G_path_layer = nx.Graph()
        nodes_path = []
        p = G_layer.nodes[entry_node_index]['pos']
        nodes_path.append((entry_node_index,{"pos": p}))

        cond = True
        while cond:
            nearest_node_current = -1
            nearest_distance_current = float('inf')
            for neihbor_i in G_layer.neighbors(current_node_index):
                vec1 = G_layer.nodes[neihbor_i]['pos']
                vec2 = G_query.nodes['Q']['pos']
                dist = np.linalg.norm( vec1 - vec2)
                if dist < nearest_distance_current:
                    nearest_node_current = neihbor_i
                    nearest_distance_current = dist
            
            if nearest_distance_current < nearest_distance_layer:
                nearest_node_layer = nearest_node_current
                nearest_distance_layer = nearest_distance_current
                nodes_path.append((nearest_node_current,{"pos": G_layer.nodes[nearest_node_current]['pos']}))
            else:
                cond = False
        
        entry_node_index = nearest_node_layer

        G_path_layer.add_nodes_from(nodes_path)
        SearchPathGraphArray.append(G_path_layer)
        EntryGraphArray.append(G_entry)

    SearchPathGraphArray.reverse()
    EntryGraphArray.reverse()
 
    return SearchPathGraphArray, EntryGraphArray

